In [1]:
%reload_ext autoreload
%autoreload 2

import torch
from tqdm.auto import tqdm


import eval
import mmlu
import utils
import t5

2023-02-02 13:27:40.606512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 13:27:41.145653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/torch/lib:/usr/local/lib/python3.8/dist-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-02 13:27:41.145695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

In [2]:
# checkpoints = ["google/flan-t5-large"]  # for testing
checkpoints = [
    "google/flan-t5-small",
    "google/flan-t5-base",
    "google/flan-t5-large",
    "google/flan-t5-xl",
    "google/flan-t5-xxl",
]


In [3]:
for checkpoint in checkpoints:
    utils.logger.info(f"Loading model from {checkpoint}.")
    m, tk = t5.load_model(checkpoint)

    for cot in [False, True]:
        dataset_results = []
        for task in tqdm(mmlu.task_list):
            config = eval.EvalConfig(
                checkpoint=checkpoint,
                dataset_name="lukaemon/mmlu",
                task_name=task,
                split="test",
                cot=cot,
                n=5,  # mmlu uses 5 shots
                cutoff=None,
                batch_size=1,
                temp=0.0,
                k=1,
                inst_fn=mmlu.task_instruction,
                cot_fn=mmlu.task_cot,
            )

            task_result = eval.eval_task(m, tk, config)
            dataset_results.append(task_result)

        eval.save_result(dataset_results)
    del m
    torch.cuda.empty_cache()


[INFO] [2023-02-02 13:27:42,270] [flan-t5] [3534233615.py:2] Loading model from google/flan-t5-small.


  0%|          | 0/57 [00:00<?, ?it/s]

[INFO] [2023-02-02 13:39:59,490] [flan-t5] [eval.py:138] Saving accuracy result to output/mmlu/accuracy/flan-t5-small_cot_False_k_1.csv.
[INFO] [2023-02-02 13:39:59,494] [flan-t5] [eval.py:148] Saving failed output to output/mmlu/failed/flan-t5-small.


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/57 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   20 │   │   │   │   cot_fn=mmlu.task_cot,                                                       │
│   21 │   │   │   )                                                                               │
│   22 │   │   │                                                                                   │
│ ❱ 23 │   │   │   task_result = eval.eval_task(m, tk, config)                                     │
│   24 │   │   │   dataset_results.append(task_result)                                             │
│   25 │   │                                                                                       │
│   26 │   │   eval.save_result(dataset_results)                                                   │
│                                                                                                  │
│ /workspaces/seed/paper/chungScalingInstructionFinetunedLanguage2022/eval.py:85 in eval_task      │
│                                                                                                  │
│    82 │   full_text_pred = []                                                                    │
│    83 │                                                                                          │
│    84 │   for batch in dl:                                                                       │
│ ❱  85 │   │   full_text_pred.extend(t5.predict(m, tk, batch, temp=config.temp, k=config.k))      │
│    86 │                                                                                          │
│    87 │   # analyze                                                                              │
│    88 │   accuracy, failed_idx, failed_output, failed_pred = analyze(                            │
│                                                                                                  │
│ /workspaces/seed/paper/chungScalingInstructionFinetunedLanguage2022/t5.py:26 in predict          │
│                                                                                                  │
│   23 │   returns: [batch_size, k], list of lists of strings                                      │
│   24 │   """                                                                                     │
│   25 │                                                                                           │
│ ❱ 26 │   batch_output = model.generate(                                                          │
│   27 │   │   input_ids=batch["input_ids"].cuda(),                                                │
│   28 │   │   attention_mask=batch["attention_mask"].cuda(),                                      │
│   29 │   │   max_length=256,                                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                 